In [47]:
from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [13]:
digits = load_digits()

In [21]:
def write_answer(number, task):
    f = open(str(task)+'.txt', 'w')
    f.write(str(number))
    f.close

#### 1. Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [30]:
clf = DecisionTreeClassifier(random_state=1)
print(cross_val_score(clf, digits.data, digits.target, cv=10).mean())
write_answer(cross_val_score(clf, digits.data, digits.target, cv=10).mean(), 1)

0.8241154562383614


#### 2. Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100. Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [31]:
clf1 = BaggingClassifier(base_estimator=clf, n_estimators=100, random_state=1)
print(cross_val_score(clf1, digits.data, digits.target, cv=10).mean())
write_answer(cross_val_score(clf1, digits.data, digits.target, cv=10).mean(), 2)

0.922054624456859


#### 3. Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на \sqrt{d} случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [62]:
clf3 = BaggingClassifier(base_estimator=clf, n_estimators=100, random_state=1, max_features=int(np.sqrt(len(digits.feature_names))))
print(cross_val_score(clf3, digits.data, digits.target, cv=10).mean())
write_answer(cross_val_score(clf3, digits.data, digits.target, cv=10).mean(), 3)

0.92820608317815


#### 4. Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же \sqrt{d}  d признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [61]:
clf_new = DecisionTreeClassifier(random_state=1,  max_features=int(np.sqrt(len(digits.feature_names))))
clf4 = BaggingClassifier(base_estimator=clf_new, n_estimators=100, random_state=1)
print(cross_val_score(clf4, digits.data, digits.target, cv=10).mean())
write_answer(cross_val_score(clf4, digits.data, digits.target, cv=10).mean(), 4)


0.9487957790192427


#### 5. Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.  На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

In [60]:
clf_RF = RandomForestClassifier(n_estimators=200, random_state=0)
print(cross_val_score(clf_RF, digits.data, digits.target, cv=10).mean())

0.9493544382371197


### 2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.


7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).

In [ ]:
2